In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/financial-sentiment-analysis/data.csv


# **Basic topics** 
* Text Cleaning
* Stemming
* Lemmatization
* Co-reference resolution
* Dependancy parsing
* POS tagging

### Text Cleaning

In [3]:
df = pd.read_csv('/kaggle/input/financial-sentiment-analysis/data.csv')
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   5842 non-null   object
 1   Sentiment  5842 non-null   object
dtypes: object(2)
memory usage: 91.4+ KB


In [5]:
df.shape

(5842, 2)

In [6]:
round((df.groupby(df['Sentiment'])['Sentence'].count()/df.shape[0])*100,2)

Sentiment
negative    14.72
neutral     53.58
positive    31.70
Name: Sentence, dtype: float64

In [7]:
text = df['Sentence'][0]+" "+df['Sentence'][1]+' abcd10@azure.com '+df['Sentence'][0]+' #'
text

"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model . $ESI on lows, down $1.50 to $2.50 BK a real possibility abcd10@azure.com The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model . #"

In [8]:
import re #importing regex library

#lower case all characters
text = text.lower()
print('------lower case all characters------')
print(text)
#removing emails
text = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", text)
print('------removing emails------')
print(text)
#removing hashtag and @ words
text= re.sub(r'@\S+', '',text) 
text= re.sub(r'#\S+', '',text)
print('------removing hashtag and @------')
print(text)
# remove urls
text = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , text) 
text= re.sub(r'www.\S+', '',text)
print('------removing urls------')
print(text)
# regex only keeps characters
text=re.sub(r"[^a-zA-Z+']", ' ',text)
print('------removing numbers------')
print(text)
#remove 's
text=re.sub(r"['’]s\b",' ',text)
print('------removing "tropic s"------')
print(text)
# regex removes repeated spaces, strip removes leading and trailing spaces
text=re.sub("\s[\s]+", " ",text).strip()
print('------removing trailing/extra spaces------')
print(text)

------lower case all characters------
the geosolutions technology will leverage benefon 's gps solutions by providing location based search technology , a communities platform , location relevant multimedia content and a new and powerful commercial model . $esi on lows, down $1.50 to $2.50 bk a real possibility abcd10@azure.com the geosolutions technology will leverage benefon 's gps solutions by providing location based search technology , a communities platform , location relevant multimedia content and a new and powerful commercial model . #
------removing emails------
the geosolutions technology will leverage benefon 's gps solutions by providing location based search technology , a communities platform , location relevant multimedia content and a new and powerful commercial model . $esi on lows, down $1.50 to $2.50 bk a real possibility  the geosolutions technology will leverage benefon 's gps solutions by providing location based search technology , a communities platform , locat

In [9]:
df['Sentence'][8]

"Kone 's net sales rose by some 14 % year-on-year in the first nine months of 2008 ."

In [10]:
def clean_1(text):
    #lower case all characters
    text = text.lower()
    #removing emails
    text = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", text)
    #removing hashtag and @ words
    text= re.sub(r'@\S+', '',text) 
    text= re.sub(r'#\S+', '',text)
    # remove urls
    text = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , text) 
    text= re.sub(r'www.\S+', '',text)
    # regex only keeps characters
    text=re.sub(r"[^a-zA-Z+']", ' ',text)
    #remove 's
    text=re.sub(r"['’]s\b",' ',text)
    # regex removes repeated spaces, strip removes leading and trailing spaces
    text=re.sub("\s[\s]+", " ",text).strip()
    return text

In [11]:
clean_1(df['Sentence'][0])

'the geosolutions technology will leverage benefon gps solutions by providing location based search technology a communities platform location relevant multimedia content and a new and powerful commercial model'

In [12]:
df['cleaned_text'] = df.Sentence.apply(lambda x: clean_1(x))

In [13]:
df.head()

,Sentence,Sentiment,cleaned_text
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta net sales d...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnish russian chamber of co...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...


In [14]:
## Stop words removingz
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

In [15]:
stopwords = list(stopwords)
print(len(stopwords))
#stopwords

326


In [16]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
print(len(stopwords))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
179


In [17]:
updated_stopwords = stopwords[0:35]

In [18]:
updated_stopwords = updated_stopwords + ['a','an','the','and','but','or']

In [19]:
## for sentiment analysis we will be using custom stopwords
df['cleaned_text2'] = df.cleaned_text.apply(lambda x: " ".join(words for words in x.split() if words not in updated_stopwords) )

In [20]:
df.head()

,Sentence,Sentiment,cleaned_text,cleaned_text2
0,The GeoSolutions technology will leverage Bene...,positive,the geosolutions technology will leverage bene...,geosolutions technology will leverage benefon ...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi on lows down to bk a real possibility,esi on lows down to bk real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,for the last quarter of componenta net sales d...,for last quarter of componenta net sales doubl...
3,According to the Finnish-Russian Chamber of Co...,neutral,according to the finnish russian chamber of co...,according to finnish russian chamber of commer...
4,The Swedish buyout firm has sold its remaining...,neutral,the swedish buyout firm has sold its remaining...,swedish buyout firm has sold remaining percent...


### Stemming

Applications in sentiment analysis, etc.

In [21]:
#Porter Stemmer
#Snowball Stemmer
#Lancaster Stemmer
#Regex Stemmer
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer, RegexpStemmer
porter = PorterStemmer()
lancaster = LancasterStemmer()
snowball = SnowballStemmer(language='english')
regexp = RegexpStemmer('ing$|s$|e$|able$',min=4)
words_list = ["friend","frienship","friends","friendships"]
print("{0:20}{1:20}{2:20}{3:30}{4:40}".format("Words",'Porter Stemmer','Snowball Stemmer','Lancaster Stemmer','Regexp Stemmer'))
for word in words_list:
    print("{0:20}{1:20}{2:20}{3:30}{4:40}".format(word,porter.stem(word),lancaster.stem(word),snowball.stem(word),regexp.stem(word)))

Words               Porter Stemmer      Snowball Stemmer    Lancaster Stemmer             Regexp Stemmer                          
friend              friend              friend              friend                        friend                                  
frienship           frienship           fri                 frienship                     frienship                               
friends             friend              friend              friend                        friend                                  
friendships         friendship          friend              friendship                    friendship                              


In [22]:
#checking stemmers for our sentiment texts
text = df['cleaned_text2'][0]+" "+df['cleaned_text2'][1]+" "+df['cleaned_text2'][2]
lancast_stem = " ".join(lancaster.stem(word) for word in text.split())
porter_stem = " ".join(porter.stem(word) for word in text.split())
snowball_stem = " ".join(snowball.stem(word) for word in text.split())
print("----------text-----------")
print(text)
print("--------------Stemmed text using porter--------------")
print(porter_stem)
print("--------------Stemmed text using lancaster--------------")
print(lancast_stem)
print("--------------Stemmed text using snowball--------------")
print(snowball_stem)

----------text-----------
geosolutions technology will leverage benefon gps solutions by providing location based search technology communities platform location relevant multimedia content new powerful commercial model esi on lows down to bk real possibility for last quarter of componenta net sales doubled to eur m from eur m for same period year earlier while moved to zero pre tax profit from pre tax loss of eur m
--------------Stemmed text using porter--------------
geosolut technolog will leverag benefon gp solut by provid locat base search technolog commun platform locat relev multimedia content new power commerci model esi on low down to bk real possibl for last quarter of componenta net sale doubl to eur m from eur m for same period year earlier while move to zero pre tax profit from pre tax loss of eur m
--------------Stemmed text using lancaster--------------
geosolv technolog wil lev benefon gps solv by provid loc bas search technolog commun platform loc relev multimed cont n

### Lemmatization

Applications in Chatbots,human-answering

In [23]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [25]:
lemmatizer = WordNetLemmatizer()
print("words :", lemmatizer.lemmatize("words")) 

words : word


In [26]:
lemmatized_text = " ".join(lemmatizer.lemmatize(word) for word in text.split())
print("----------text-----------")
print(text)
print("----------Lemmatized text-----------")
print(lemmatized_text)

----------text-----------
geosolutions technology will leverage benefon gps solutions by providing location based search technology communities platform location relevant multimedia content new powerful commercial model esi on lows down to bk real possibility for last quarter of componenta net sales doubled to eur m from eur m for same period year earlier while moved to zero pre tax profit from pre tax loss of eur m
----------Lemmatized text-----------
geosolutions technology will leverage benefon gps solution by providing location based search technology community platform location relevant multimedia content new powerful commercial model esi on low down to bk real possibility for last quarter of componenta net sale doubled to eur m from eur m for same period year earlier while moved to zero pre tax profit from pre tax loss of eur m


### POS Tagging

* POS tag means parts of speech tagging.
* By default pos tag in lemmatization is noun.


In [27]:
#tag function to return tag of words 
def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [28]:
sentence = "Cause the source file is compressed and you need to uncompressed it"
nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), nltk_tagged)
print(nltk_tagged)
print('--------------------------------------')
for word, tag in wordnet_tagged:
    print(word,tag)

[('Cause', 'IN'), ('the', 'DT'), ('source', 'NN'), ('file', 'NN'), ('is', 'VBZ'), ('compressed', 'VBN'), ('and', 'CC'), ('you', 'PRP'), ('need', 'VBP'), ('to', 'TO'), ('uncompressed', 'VB'), ('it', 'PRP')]
--------------------------------------
Cause None
the None
source n
file n
is v
compressed v
and None
you None
need v
to None
uncompressed v
it None


In [29]:
#Lemmatization function
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), nltk_tagged)
    lemmatized_tokens = []
    
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_tokens.append(word)
        else:        
            lemmatized_tokens.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_tokens

In [30]:
print("-----------------original text--------------")
print(text)
print('-------------lemmatization without POS-----------')
print(lemmatized_text)
print('---------------Lemmatization with POS Tagging-----------')
" ".join(word for word in lemmatize_sentence(text))

-----------------original text--------------
geosolutions technology will leverage benefon gps solutions by providing location based search technology communities platform location relevant multimedia content new powerful commercial model esi on lows down to bk real possibility for last quarter of componenta net sales doubled to eur m from eur m for same period year earlier while moved to zero pre tax profit from pre tax loss of eur m
-------------lemmatization without POS-----------
geosolutions technology will leverage benefon gps solution by providing location based search technology community platform location relevant multimedia content new powerful commercial model esi on low down to bk real possibility for last quarter of componenta net sale doubled to eur m from eur m for same period year earlier while moved to zero pre tax profit from pre tax loss of eur m
---------------Lemmatization with POS Tagging-----------


'geosolutions technology will leverage benefon gps solution by provide location base search technology community platform location relevant multimedia content new powerful commercial model esi on low down to bk real possibility for last quarter of componenta net sale double to eur m from eur m for same period year earlier while move to zero pre tax profit from pre tax loss of eur m'

### **Co-reference Resolution**

### Pre-processing in Coreference resolution
Coreference resolution requires a pre-processing pipeline that consists of a variety of NLP tasks. It may includes

* **Named Entity Recognition**: which finds and classifies named entities in a text into predefined categories such as locations, organizations or names of persons.

* **Entity Linking or Wikification**: which aligns textual mentions of named entities to their corresponding entries in a knowledge base.

* **Part-of-Speech Tagging**: a process of assigning parts of speech to words in a sentence.

* **Stemming**: a process of cutting off the beginning or end of the word, using a list of common prefixes and suffixes that can be found in an inflected word.

### When do we use Coreference Resolution?
Coreference resolution is using in a variety of NLP tasks such as,
* Text understanding
* Document summarization
* Information extraction
* Sentiment analysis
* Machine translation

### Implementation
* **spaCy-huggingface(NeuralCoref) coreference resolution**
NeuralCoref is a pipeline extension for spaCy 2.1+ which annotates and resolves coreference clusters using a neural network. NeuralCoref is production-ready, integrated in spaCy’s NLP pipeline and extensible to new training datasets.

In [49]:
import spacy
print(spacy.__version__)

3.7.2


In [51]:
!pip install /Users/you/en_core_web_sm-2.0.12.tar.gz

Processing /Users/you/en_core_web_sm-2.0.12.tar.gz
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/Users/you/en_core_web_sm-2.0.12.tar.gz'



In [50]:
!pip uninstall -y spacy

Found existing installation: spacy 3.7.2
Uninstalling spacy-3.7.2:
  Successfully uninstalled spacy-3.7.2


In [48]:
!pip install -U spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/d6/c8/d659de66da2be05ad10fe029954a90a5fb6ddffb75aaab6702173e9dd0c9/spacy-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 68.1 MB/s eta 0:00:00:00:010:01


In [1]:
!pip install fastcoref

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for datasets>=2.5.2 from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.5 MB/s eta 0:00:00
  Created wheel for fastcoref: filename=fastcoref-2.1.6-py3-none-any.whl size=31253 sha256=e35a04db757f29346a55987ec9dc60a71e3216a3476bbdaa3105

In [2]:
from fastcoref import FCoref
model = FCoref()

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/362M [00:00<?, ?B/s]

In [10]:
text = ['Fastcoref is a great package. I am glad Noa introduced it to me.',"Joseph Robinette Biden Jr. is an American politician who is the 46th and\
current president of the United States. A member of the Democratic Party, \
he served as the 47th vice president from 2009 to 2017 under Barack Obama and\
represented Delaware in the United States Senate from 1973 to 2009."]
preds = model.predict(texts=text)

print(preds[0].get_clusters())
print(preds[1].get_clusters())

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Inference:   0%|          | 0/2 [00:00<?, ?it/s]

[['Fastcoref', 'it'], ['I', 'me']]
[['Joseph Robinette Biden Jr.', 'he'], ['2009', '2009']]


In [17]:
from fastcoref import spacy_component
import spacy

text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

doc = nlp(text)
doc._.coref_clusters

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[[(0, 5), (39, 42), (79, 82)]]

### Dependancy Parsing